In [ ]:
from tensorflow.keras import layers, models, Model
import numpy as np
import pandas as pd

In [25]:
# Import the data
path = 'Project3database.csv'
df = pd.read_csv(path)
df.head()

,Year,Tm,#Bat,BatAge,R/G,G,PA,AB,RS,H,...,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,Playoff,Champion
0,2023,Arizona Diamondbacks,54,27.4,4.60,162,6124,5436,746,1359,...,56,134,0.990,43,4,46.0,0.0,9.0,1,0
1,2023,Atlanta Braves,53,27.9,5.85,162,6249,5597,947,1543,...,82,133,0.986,21,2,16.0,0.0,2.0,1,0
2,2023,Baltimore Orioles,50,27.2,4.98,162,6123,5495,807,1399,...,71,140,0.988,37,3,34.0,0.0,-3.0,1,0
3,2023,Boston Red Sox,56,28.6,4.77,162,6174,5562,772,1437,...,102,137,0.982,-17,-2,-20.0,-2.0,-2.0,0,0
4,2023,Chicago Cubs,48,28.4,5.06,162,6220,5504,819,1399,...,92,137,0.984,9,1,36.0,0.0,1.0,0,0


In [26]:
#Display basic information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 83 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Year      720 non-null    int64  
 1   Tm        720 non-null    object 
 2   #Bat      720 non-null    int64  
 3   BatAge    720 non-null    float64
 4   R/G       720 non-null    float64
 5   G         720 non-null    int64  
 6   PA        720 non-null    int64  
 7   AB        720 non-null    int64  
 8   RS        720 non-null    int64  
 9   H         720 non-null    int64  
 10  2B        720 non-null    int64  
 11  3B        720 non-null    int64  
 12  HR        720 non-null    int64  
 13  RBI       720 non-null    int64  
 14  SB        720 non-null    int64  
 15  CS        720 non-null    int64  
 16  BB        720 non-null    int64  
 17  SO        720 non-null    int64  
 18  BA        720 non-null    float64
 19  OBP       720 non-null    float64
 20  SLG       720 non-null    float6

In [27]:
#Drop columns
clean_df = df.drop(['Tm','Rdrs/yr','Rgood','Rdrs'], axis=1)
clean_df

,Year,#Bat,BatAge,R/G,G,PA,AB,RS,H,2B,...,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Playoff,Champion
0,2023,54,27.4,4.60,162,6124,5436,746,1359,274,...,5761,4306,1399,56,134,0.990,43,4,1,0
1,2023,53,27.9,5.85,162,6249,5597,947,1543,293,...,5741,4320,1339,82,133,0.986,21,2,1,0
2,2023,50,27.2,4.98,162,6123,5495,807,1399,309,...,5876,4361,1444,71,140,0.988,37,3,1,0
3,2023,56,28.6,4.77,162,6174,5562,772,1437,339,...,5821,4290,1429,102,137,0.982,-17,-2,0,0
4,2023,48,28.4,5.06,162,6220,5504,819,1399,269,...,5888,4306,1490,92,137,0.984,9,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,2000,36,29.9,5.71,162,6418,5519,925,1535,304,...,6070,4333,1644,93,173,0.985,6,1,1,0
716,2000,46,29.6,5.48,162,6369,5478,887,1481,259,...,5936,4301,1524,111,148,0.981,43,4,1,0
717,2000,51,30.1,4.55,161,6206,5505,733,1414,253,...,6226,4294,1814,118,169,0.981,10,1,0,0
718,2000,42,29.5,5.23,162,6364,5648,848,1601,330,...,6016,4287,1594,135,162,0.978,-62,-6,0,0


In [28]:
#Drop null values
clean_df.dropna()

,Year,#Bat,BatAge,R/G,G,PA,AB,RS,H,2B,...,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Playoff,Champion
0,2023,54,27.4,4.60,162,6124,5436,746,1359,274,...,5761,4306,1399,56,134,0.990,43,4,1,0
1,2023,53,27.9,5.85,162,6249,5597,947,1543,293,...,5741,4320,1339,82,133,0.986,21,2,1,0
2,2023,50,27.2,4.98,162,6123,5495,807,1399,309,...,5876,4361,1444,71,140,0.988,37,3,1,0
3,2023,56,28.6,4.77,162,6174,5562,772,1437,339,...,5821,4290,1429,102,137,0.982,-17,-2,0,0
4,2023,48,28.4,5.06,162,6220,5504,819,1399,269,...,5888,4306,1490,92,137,0.984,9,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,2000,36,29.9,5.71,162,6418,5519,925,1535,304,...,6070,4333,1644,93,173,0.985,6,1,1,0
716,2000,46,29.6,5.48,162,6369,5478,887,1481,259,...,5936,4301,1524,111,148,0.981,43,4,1,0
717,2000,51,30.1,4.55,161,6206,5505,733,1414,253,...,6226,4294,1814,118,169,0.981,10,1,0,0
718,2000,42,29.5,5.23,162,6364,5648,848,1601,330,...,6016,4287,1594,135,162,0.978,-62,-6,0,0


In [52]:
#Describe the data
# display all columns
pd.set_option('display.max_columns', None)
print(clean_df.describe())

              Year        #Bat      BatAge         R/G           G  \
count   720.000000  720.000000  720.000000  720.000000  720.000000   
mean   2011.500000   47.930556   28.660278    4.564194  157.708333   
std       6.926999    6.065535    1.307156    0.510573   20.417913   
min    2000.000000   34.000000   25.400000    3.170000   58.000000   
25%    2005.750000   44.000000   27.800000    4.220000  162.000000   
50%    2011.500000   47.000000   28.600000    4.540000  162.000000   
75%    2017.250000   51.000000   29.500000    4.900000  162.000000   
max    2023.000000   69.000000   33.500000    6.040000  163.000000   

                PA           AB          RS            H          2B  \
count   720.000000   720.000000  720.000000   720.000000  720.000000   
mean   6029.109722  5378.838889  719.450000  1384.966667  276.601389   
std     804.285879   716.705082  122.748055   207.974680   47.398623   
min    2011.000000  1752.000000  219.000000   390.000000   73.000000   
25%    60

In [42]:
#Get mean value of dataset for baseline
mean_df = clean_df.mean().to_frame().T
mean_df

,Year,#Bat,BatAge,R/G,G,PA,AB,RS,H,2B,...,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Playoff,Champion
0,2011.5,47.930556,28.660278,4.564194,157.708333,6029.109722,5378.838889,719.45,1384.966667,276.601389,...,5855.290278,4215.283333,1544.1875,95.819444,141.241667,0.983635,1.076389,0.119444,0.3125,0.033333


In [32]:
#Get a dataframe for teams that made the playoffs for comparison to leverage statistics with the most deviation
df_playoffs = clean_df[clean_df['Playoff'] == 1]
df_playoffs

,Year,#Bat,BatAge,R/G,G,PA,AB,RS,H,2B,...,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Playoff,Champion
0,2023,54,27.4,4.60,162,6124,5436,746,1359,274,...,5761,4306,1399,56,134,0.990,43,4,1,0
1,2023,53,27.9,5.85,162,6249,5597,947,1543,293,...,5741,4320,1339,82,133,0.986,21,2,1,0
2,2023,50,27.2,4.98,162,6123,5495,807,1399,309,...,5876,4361,1444,71,140,0.988,37,3,1,0
10,2023,41,28.8,5.10,162,6253,5567,827,1441,280,...,5763,4336,1346,81,144,0.986,33,3,1,0
13,2023,58,30.9,5.59,162,6333,5524,906,1422,303,...,5821,4339,1406,76,139,0.987,42,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,2000,46,31.4,5.41,161,6311,5556,871,1541,294,...,5869,4273,1487,109,132,0.981,-22,-2,1,0
710,2000,44,27.3,5.88,161,6432,5560,947,1501,281,...,6166,4306,1726,134,164,0.978,-46,-4,1,0
714,2000,37,31.3,5.60,162,6444,5497,907,1481,300,...,6053,4325,1629,99,176,0.984,29,3,1,0
715,2000,36,29.9,5.71,162,6418,5519,925,1535,304,...,6070,4333,1644,93,173,0.985,6,1,1,0


In [36]:
#Describe df_playoffs
df_playoffs.describe()

,Year,#Bat,BatAge,R/G,G,PA,AB,RS,H,2B,...,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Playoff,Champion
count,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,...,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.0,225.000000
mean,2012.511111,46.400000,28.946222,4.872444,154.733333,5959.995556,5284.742222,754.711111,1383.182222,277.617778,...,5733.204444,4152.542222,1492.831111,87.831111,133.804444,0.984671,16.755556,1.666667,1.0,0.106667
std,6.977714,5.652907,1.330010,0.470047,26.306572,1043.777260,927.888316,148.945553,265.933728,57.423051,...,1021.501168,722.631791,305.505842,21.456308,31.125279,0.002507,34.579073,3.286879,0.0,0.309377
min,2000.000000,34.000000,25.900000,3.820000,58.000000,2011.000000,1752.000000,240.000000,390.000000,73.000000,...,1941.000000,1419.000000,442.000000,20.000000,33.000000,0.976000,-87.000000,-8.000000,1.0,0.000000
25%,2007.000000,42.000000,28.000000,4.510000,162.000000,6152.000000,5484.000000,724.000000,1384.000000,271.000000,...,5849.000000,4323.000000,1427.000000,77.000000,124.000000,0.983000,-3.000000,0.000000,1.0,0.000000
50%,2013.000000,46.000000,28.800000,4.850000,162.000000,6232.000000,5528.000000,780.000000,1441.000000,288.000000,...,6004.000000,4347.000000,1568.000000,89.000000,137.000000,0.985000,20.000000,2.000000,1.0,0.000000
75%,2019.000000,50.000000,29.700000,5.220000,162.000000,6302.000000,5583.000000,829.000000,1497.000000,305.000000,...,6116.000000,4373.000000,1655.000000,102.000000,153.000000,0.986000,38.000000,4.000000,1.0,0.000000
max,2023.000000,61.000000,32.900000,6.040000,163.000000,6537.000000,5769.000000,978.000000,1667.000000,373.000000,...,6459.000000,4469.000000,1951.000000,134.000000,196.000000,0.991000,104.000000,9.000000,1.0,1.000000


In [43]:
#Get mean values for playoff dataframe 
mean_playoff_df = df_playoffs.mean().to_frame().T
mean_playoff_df.head()

,Year,#Bat,BatAge,R/G,G,PA,AB,RS,H,2B,...,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Playoff,Champion
0,2012.511111,46.4,28.946222,4.872444,154.733333,5959.995556,5284.742222,754.711111,1383.182222,277.617778,...,5733.204444,4152.542222,1492.831111,87.831111,133.804444,0.984671,16.755556,1.666667,1.0,0.106667


In [51]:
#Combie both mean dataframes and create a column to display the differnces between the two
mean_combined = pd.concat([mean_df, mean_playoff_df], ignore_index=True)

# display all columns
pd.set_option('display.max_columns', None)
print(mean_combined)


          Year       #Bat     BatAge       R/G           G           PA  \
0  2011.500000  47.930556  28.660278  4.564194  157.708333  6029.109722   
1  2012.511111  46.400000  28.946222  4.872444  154.733333  5959.995556   

            AB          RS            H          2B         3B          HR  \
0  5378.838889  719.450000  1384.966667  276.601389  27.866667  171.402778   
1  5284.742222  754.711111  1383.182222  277.617778  26.444444  184.600000   

          RBI         SB         CS          BB           SO       BA  \
0  685.690278  89.294444  34.209722  508.033333  1165.368056  0.25699   
1  720.328889  88.026667  31.520000  533.893333  1140.968889  0.26064   

        OBP       SLG       OPS        OPS+           TB         GDP  \
0  0.325478  0.414703  0.740171   97.177778  2231.509722  119.925000   
1  0.333173  0.428520  0.761707  103.075556  2267.488889  118.364444   

         HBP         SH         SF        IBB          LOB         #P  \
0  57.412500  41.768056  41.9

In [47]:
# Calculating the difference between consecutive rows
df_diff = mean_combined.diff()

# Selecting the second row to get the difference between the first and second row
difference = df_diff.iloc[1].to_frame().T
difference

,Year,#Bat,BatAge,R/G,G,PA,AB,RS,H,2B,...,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Playoff,Champion
1,1.011111,-1.530556,0.285944,0.30825,-2.975,-69.114167,-94.096667,35.261111,-1.784444,1.016389,...,-122.085833,-62.741111,-51.356389,-7.988333,-7.437222,0.001036,15.679167,1.547222,0.6875,0.073333


In [ ]:
# Preprocess y
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Preprocess "quality" column (one-hot encoding)
quality_encoder = OneHotEncoder(sparse_output=False)
quality_encoded = quality_encoder.fit_transform(df[['quality']])
quality_columns = quality_encoder.get_feature_names_out(['quality'])
df_quality_encoded = pd.DataFrame(quality_encoded, columns=quality_columns)

# Preprocess "color" column (label encoding for binary; one-hot encoding for multiple categories)
color_encoder = LabelEncoder()
df['color_encoded'] = color_encoder.fit_transform(df['color'])

# Concatenate the encoded columns to the original DataFrame
df_processed = pd.concat([df, df_quality_encoded], axis=1)

# Drop the original "quality" and "color" columns
df_processed = df_processed.drop(['quality', 'color'], axis=1)

df_processed.head()

In [ ]:
# Split data into X and two separate y variables
X = df_processed.drop(columns=['quality_good', 'quality_ok', 'quality_bad', 'color_encoded'])

y_color = df_processed['color_encoded']

y_quality = df_processed[['quality_good', 'quality_ok', 'quality_bad']]

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_color_train, y_color_test, y_quality_train, y_quality_test = train_test_split(X, y_color, y_quality)

In [ ]:
# Create the shared layers of the model

# Input layer
input_layer = layers.Input(shape=(X.shape[1],), name='input_features')

# Shared hidden layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [ ]:
# Branch for quality prediction
quality_output = layers.Dense(3, activation='softmax', name='quality_output')(shared_layer2)

# Branch for color prediction
color_output = layers.Dense(1, activation='sigmoid', name='color_output')(shared_layer2)

In [ ]:
# Create the model
model = Model(inputs=input_layer, outputs=[quality_output, color_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'quality_output': 'categorical_crossentropy', 'color_output': 'binary_crossentropy'},
              metrics={'quality_output': 'accuracy', 'color_output': 'accuracy'})

# Display the model summary
model.summary()

In [ ]:
# Fit the model
model.fit(
    X,
    {'quality_output': y_quality, 'color_output': y_color},
    epochs=10,
    batch_size=32,
    validation_split=0.2
)

In [ ]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test, {'quality_output': y_quality_test, 'color_output': y_color_test})
test_results

In [ ]:
# Print the quality and color accuracy
print(f"Quality Accuracy: {test_results[3]}")
print(f"Color Accuracy: {test_results[4]}")